In [91]:
import os 
import pandas as pd 
import itertools
import re
import numpy as np

In [20]:
def preprocess(data, info):
    a = data.copy()
    info = re.findall(r".*(?=.csv)",info)[0]
    a = a.rename(columns={"First Tooltip": info})

    """    
    if ("Dim1" in a.columns.values):
        a['Both sexes'] = (a['Dim1'] == "Both sexes").replace({False:0, True:1})
        a['Male'] = (a['Dim1'] == "Male").replace({False:0, True:1})
        a['Female'] = (a['Dim1'] == "Female").replace({False:0, True:1})
    else:
        a['Both sexes'] = (a['Location'] != "Both sexes").replace({False:0, True:1})
        a['Male'] = (a['Location'] != "Male").replace({False:0, True:1})
        a['Female'] = (a['Location'] != "Female").replace({False:0, True:1})
    """

     
    data_2015 = a[a["Period"] == 2015]
    
    data_2016 = a[a["Period"] == 2016]
    
    data_2017 = a[a["Period"] == 2017]
    
    data_2018 = a[a["Period"] == 2018]
    
    data_2019 = a[a["Period"] == 2019]
    
    data_2020 = a[a["Period"] == 2020]
    
    data_2021 = a[a["Period"] == 2021]

    year_dic = {'2015':data_2015,'2016':data_2016,'2017':data_2017,'2018':data_2018,
                '2019':data_2019,'2020':data_2020,'2021':data_2021}
    
    if("Dim1" in a.columns.values):
        for key,df in year_dic.items():
            temp = df.groupby("Location").mean()
            temp.reset_index(inplace=True)
            temp = temp.loc[:, ['Location', info]]
            year_dic[key] = temp
    
    else:
        for key,df in year_dic.items():
            temp = df.loc[:, ['Location', info]]
            year_dic[key] = temp
    return year_dic


In [89]:
def merge_dic(data_lst):
    year_lst = [str(i) for i in range(2015,2022)]
    result = []
    for year in year_lst:
        temp = None
        indicator = False
        for variable_dict in data_lst: 
            if indicator == False:
                temp = variable_dict[year]
                indicator = True
            else:          
                temp = pd.merge(temp, variable_dict[year], on = ["Location"])
        temp['Period'] = np.array([year]*temp.shape[0])
        result.append(temp)
    return result

In [ ]:
raw_data = read_data()
test = []
comb = []
result_lst = []

data_lst = raw_data
data_lst = run_proprocess(data_lst)
year_lst = [str(i) for i in range(2015,2022)]
result = []
for year in year_lst:
    temp = None

In [4]:
def read_data():
    data_lst = []

    for info in os.listdir("fill_data"): 

        if info == ".DS_Store":
            continue
        domain = os.path.abspath(r"fill_data")
        dirc = os.path.join(domain,info)
        data = pd.read_csv(dirc)
        
        data_lst.append([data, info])
    return data_lst


In [5]:
def run_proprocess(data_lst):
    result = []
    for [data, info] in data_lst:
        temp = preprocess(data, info)
        result.append(temp)
    return result

In [114]:
def main():
    raw_data = read_data()
    #[how many dataset uses, how many useful data]
    test = []
    
    #dataset (doesn't merge)
    comb = []
    
    #dataset merge
    result_lst = []
    for L in range(0, len(raw_data)+1):
        for subset in itertools.combinations(raw_data, L):
            if(len(subset) != 0):
                data_lst = subset
                data_lst = run_proprocess(data_lst)
                result = merge_dic(data_lst)
                count = 0
                for i in result:
                    count += i.shape[0]

                test.append([len(subset), count])

                temp = pd.DataFrame()
                for i in range(len(result)):
                    temp = pd.concat([temp, result[i]])
                
                
                result_lst.append(temp)
                comb.append(subset)
                
                
                
    return [test,comb,result_lst]


In [167]:
#don't run it if there are more than 15 datasets in train
[test, comb, result] = main()


In [168]:
happiness_score = pd.DataFrame()
for info in os.listdir("world-happiness-report"): 
    print(info)
    if info == ".DS_Store":
        continue
    domain = os.path.abspath(r"world-happiness-report")
    dirc = os.path.join(domain,info)
    data = pd.read_csv(dirc)
    data = data[['Country', 'Happiness Score']]
    info = re.findall(r".*(?=.csv)",info)[0]
    data["Period"] = np.array([info]*data.shape[0])
    data = data.rename(columns={"Country": "Location"})
    happiness_score = pd.concat([happiness_score, data])


.DS_Store
2020.csv
2021.csv
2019.csv
2018.csv
2015.csv
2016.csv
2017.csv


In [173]:

all_data = result[-1]
all_data = pd.merge(all_data, happiness_score, on = ["Location", "Period"])

all_data

,Location,basicDrinkingWaterServices,alcoholSubstanceAbuse,30-70cancerChdEtc,basicHandWashing,atLeastBasicSanitizationServices,Period,Happiness Score
0,Afghanistan,52.388994,0.204667,30.005417,43.397952,44.485110,2015,3.575
1,Algeria,88.449639,0.878873,14.317035,81.503736,86.314097,2015,5.605
2,Angola,26.712737,7.881962,16.817750,24.519619,43.703482,2015,4.033
3,Armenia,99.416965,6.058881,23.443990,91.908775,91.525044,2015,4.350
4,Azerbaijan,79.457117,3.907791,22.741715,83.240000,91.088383,2015,5.212
...,...,...,...,...,...,...,...,...
471,Turkmenistan,103.800627,5.166102,26.198997,107.090532,101.327282,2021,5.066
472,Uganda,47.637959,16.650816,21.109981,29.043015,19.947779,2021,4.636
473,Yemen,65.284422,0.124719,29.168455,52.932831,66.564103,2021,3.658
474,Zambia,44.172312,7.732305,16.589173,15.051012,27.104972,2021,4.073


In [175]:
out_dirc = "all_data.csv"

all_data.to_csv(out_dirc,sep=',',index=True,header=True)